In [82]:
with open("input.txt") as file:
    lines = file.read().splitlines()


def parse_button(button):
    bitmap = 0
    for x in button[1:-1].split(","):
        bitmap |= 1 << int(x)
    return bitmap


def parse_line(line):
    line = line.split()
    lights = int(line[0][1:-1].replace("#", "1").replace(".", "0")[::-1], 2)
    buttons = [parse_button(b) for b in line[1:-1]]
    joltage = [int(x) for x in line[-1][1:-1].split(",")]
    return lights, buttons, joltage


parsed_lines = [parse_line(line) for line in lines]

In [83]:
# per button either press or dont. no multiple necessary
total = 0

for lights, buttons, _ in parsed_lines:
    min_presses = int(1e9)
    for button_mask in range(1 << len(buttons)):
        current_lights = lights
        for i in range(len(buttons)):
            if (button_mask >> i) & 1:
                current_lights ^= buttons[i]
        if current_lights == 0:
            min_presses = min(min_presses, bin(button_mask).count("1"))
    total += min_presses
total

428

In [85]:
import numpy as np
from scipy.optimize import milp, LinearConstraint, Bounds

total = 0

for _, buttons, joltage in parsed_lines:
    num_jolts = len(joltage)
    buttons = (
        np.array([list(format(b, f"0{num_jolts}b")[::-1]) for b in buttons])
        .astype(int)
        .T
    )
    joltage = np.array(joltage).T
    # presses = np.linalg.lstsq(buttons, joltage) # only gives real solutions -> integer programming
    bounds = Bounds(lb=0, ub=np.inf)
    constraints = LinearConstraint(buttons, joltage, joltage)
    integrality = np.ones(buttons.shape[1])
    c = np.ones(buttons.shape[1])
    res = milp(c=c, constraints=constraints, bounds=bounds, integrality=integrality)
    if not res.success:
        print("No solution found!")
        continue
    presses = np.round(res.x).astype(int).sum()
    total += presses
int(total)

16613